1.1. **(5 поени)** Објаснете како работи алфа-бета поткаструвањето.

1.2. **(25 поени)** Алгоритамот минимакс со алфа-бета поткастрување, каков што го учевме на часовите, е рекурзивен. Претворете го во итеративен (нерекурзивен). Имплементирајте го решението во Пајтон. Патем, внимавајте алгоритамот да го враќа најдобриот потег како решение.

In [11]:
class Graph:
    def __init__(self):
        self.adj = {}

    def add_vertex(self, v):
        if v not in self.adj:
            self.adj[v] = []

    def add_edge(self, edge, add_reversed=False):
        u, v = edge
        self.adj[u].append(v)
        if add_reversed:
            self.adj[v].append(u)

    def neighbours(self, v):
        return self.adj.get(v, [])

In [12]:
g = Graph()
for vertex in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
               'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V']:
    g.add_vertex(vertex)
for edge in [('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'E'), ('B', 'F'), ('C', 'G'), 
             ('C', 'H'), ('D', 'I'), ('D', 'J'), ('E', 'K'), ('E', 'L'),
             ('F', 'M'), ('F', 'N'), ('G', 'O'), ('G', 'P'), ('H', 'Q'), ('H', 'R'),
             ('I', 'S'), ('I', 'T'), ('J', 'U'), ('J', 'V')]:
    g.add_edge(edge, add_reversed=False)
leaves = {'K': 4, 'L': 6, 'M': 7, 'N': 9, 'O': 1, 'P': 2, 'Q': 0, 'R': 1, 'S': 8, 'T': 1, 'U': 9, 'V': 2}

In [13]:
plus_inf = float('inf')
minus_inf = float('-inf')

def other_player(player):
    return 'MIN' if player == 'MAX' else 'MAX'


def iterative_minimax_alpha_beta(graph, start_node, start_player):
    stack = []

    stack.append({
        "node": start_node,
        "player": start_player,
        "alpha": minus_inf,
        "beta": plus_inf,
        "best": plus_inf if start_player == 'MIN' else minus_inf,
        "children": None,
        "child_index": 0,
        "best_move": None
    })

    while stack:
        frame = stack[-1]
        node = frame["node"]
        player = frame["player"]

        # 1. Ако сме во лист
        if node in leaves:
            value = leaves[node]
            stack.pop()

            if not stack:
                return value, None

            parent = stack[-1]
            update_parent(parent, value, node)
            continue

        # 2. Ако првпат сме на јазолот → земи деца
        if frame["children"] is None:
            frame["children"] = list(graph.neighbours(node))

        # 3. Ако нема повеќе деца → враќаме best
        if frame["child_index"] >= len(frame["children"]):
            value = frame["best"]
            stack.pop()

            if not stack:
                return value, frame["best_move"]

            parent = stack[-1]
            update_parent(parent, value, node)
            continue

        # 4. Alpha-beta поткастрување
        if frame["alpha"] >= frame["beta"]:
            value = frame["best"]
            stack.pop()

            if not stack:
                return value, frame["best_move"]

            parent = stack[-1]
            update_parent(parent, value, node)
            continue

        # 5. Оди на следно дете
        child = frame["children"][frame["child_index"]]
        frame["child_index"] += 1

        child_player = other_player(player)

        stack.append({
            "node": child,
            "player": child_player,
            "alpha": frame["alpha"],
            "beta": frame["beta"],
            "best": plus_inf if child_player == 'MIN' else minus_inf,
            "children": None,
            "child_index": 0,
            "best_move": None
        })


def update_parent(parent, value, child_node):
    if parent["player"] == 'MIN':
        if value < parent["best"]:
            parent["best"] = value
        parent["beta"] = min(parent["beta"], value)
    else:  # MAX
        if value > parent["best"]:
            parent["best"] = value
            parent["best_move"] = child_node
        parent["alpha"] = max(parent["alpha"], value)

In [14]:
value, best_move = iterative_minimax_alpha_beta(g, 'A', 'MAX')
value, best_move

(8, 'D')

1.3. **(10 поени)** Дали е оправдано давањето првенство на потезите при минимакс пребарувањето? На пример, кај играта Поврзи 4, работена на вежби, има 7 колони во кои може да се постави жетонот. При пишување на функцијата expand_state(), сосема е логично да се направи expand_state() да ги врати наредните потези (каде се пуштааат жетоните) по овој редослед: [колона1, колона2, колона3, колона4, колона5, колона6, колона7]. Толкувајте дали алгоритамот би работел подобро со поинаков редослед на потезите .